# Cross Validation
Random Forest.

Mouse, Bowtie, transcripts.

Stats from Filtered.bam files.

In [1]:
from datetime import datetime
print(datetime.now())

2023-11-16 12:57:17.853823


In [2]:
import tensorflow as tf
dt='float32'
tf.keras.backend.set_floatx('float32')
tf.random.set_seed(42) # supposedly leads to reproducible results

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

try:
    from google.colab import drive
    IN_COLAB = True
    print('Running on CoLab')
    PATH='/content/drive/'
    drive.mount(PATH)
    DATA_DIR=PATH+'My Drive/data/IRP2/Mouse/Bowtie/'  # must end in "/"
    MODEL_DIR=PATH+'My Drive/data/IRP2/Models/'  # must end in "/"
except:
    IN_COLAB = False
    print('Running on Mac')
    DATA_DIR="Mouse/Bowtie/"
    MODEL_DIR="Models/"
SAVE_MODEL_FILENAME = None

GPU device not found
Running on CoLab
Mounted at /content/drive/


In [3]:
from platform import python_version
print('Python',python_version())
import random
import numpy as np
np.random.seed(42) # supposedly sets scikit-learn
import pandas as pd  # for plotting
import time # sleep function
from os.path import isfile
import gzip
from matplotlib import pyplot as plt
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
from sklearn.model_selection import KFold

from tensorflow import keras
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RFC

EPOCHS=150

Python 3.10.12
sklearn 1.2.2


## Data Load

In [4]:
MAX_LINES_TO_LOAD =    1000000 # training - 1M lines requires 2GB RAM
#MAX_LINES_TO_LOAD =    10000 # use this for debugging

VALID_PORTION = 0.20

DATA_FILE_0 = 'filtered.B6_read_stats.csv.gz'
DATA_FILE_1 = 'filtered.D2_read_stats.csv.gz'

print('Data directory: %s'%DATA_DIR)
print('Data file 0 %s'%DATA_FILE_0)
print('Data file 1 %s'%DATA_FILE_1)
print('Input lines for training: %d'%MAX_LINES_TO_LOAD)

Data directory: /content/drive/My Drive/data/IRP2/Mouse/Bowtie/
Data file 0 filtered.B6_read_stats.csv.gz
Data file 1 filtered.D2_read_stats.csv.gz
Input lines for training: 1000000


In [5]:
# P1 parent 1
# R1 read 1
# PS primary or secondary
# AS bowtie alignment score (0 is best)
# ED edit distance
# MM mismatch count
# GO gap open count
# GE gap extend count
feature_names = [
    'P1 R1 AS',
    'P1 R1 ED',
    ##'P1 R1 MAT',
    'P1 R1 MM',
    'P1 R1 HQMM',
    'P1 R1 GO',
    'P1 R1 GE',
    'P1 R1 INS',
    'P1 R1 DELS',
    'P1 R1 HQINS',
    'P1 R1 HQDEL',
    'P1 R2 AS',
    'P1 R2 ED',
    ##'P1 R2 MAT',
    'P1 R2 MM',
    'P1 R2 HQMM',
    'P1 R2 GO',
    'P1 R2 GE',
    'P1 R2 INS',
    'P1 R2 DELS',
    'P1 R2 HQINS',
    'P1 R2 HQDEL',
    'P2 R1 AS',
    'P2 R1 ED',
    ##'P2 R1 MAT',
    'P2 R1 MM',
    'P2 R1 HQMM',
    'P2 R1 GO',
    'P2 R1 GE',
    'P2 R1 INS',
    'P2 R1 DELS',
    'P2 R1 HQINS',
    'P2 R1 HQDEL',
    'P2 R2 AS',
    'P2 R2 ED',
    ##'P2 R2 MAT',
    'P2 R2 MM',
    'P2 R2 HQMM',
    'P2 R2 GO',
    'P2 R2 GE',
    'P2 R2 INS',
    'P2 R2 DELS',
    'P2 R2 HQINS',
    'P2 R2 HQDEL',
    ##'R1 length',
    ##'R2 length',
    ##'P1 span',
    ##'P2 span',
    'Span diff',
    'AS diff',
    'ED diff',
    'MAT diff',
    'MM diff',
    'HQMM diff',
    'GO diff',
    'GE diff',
    'INS diff',
    'DELS diff',
    'HQINS diff',
    'HQDEL diff',
    'PARENT']
print('Total feature names: ',len(feature_names))
for i in range(len(feature_names)):
    print(i,feature_names[i])

Total feature names:  53
0 P1 R1 AS
1 P1 R1 ED
2 P1 R1 MM
3 P1 R1 HQMM
4 P1 R1 GO
5 P1 R1 GE
6 P1 R1 INS
7 P1 R1 DELS
8 P1 R1 HQINS
9 P1 R1 HQDEL
10 P1 R2 AS
11 P1 R2 ED
12 P1 R2 MM
13 P1 R2 HQMM
14 P1 R2 GO
15 P1 R2 GE
16 P1 R2 INS
17 P1 R2 DELS
18 P1 R2 HQINS
19 P1 R2 HQDEL
20 P2 R1 AS
21 P2 R1 ED
22 P2 R1 MM
23 P2 R1 HQMM
24 P2 R1 GO
25 P2 R1 GE
26 P2 R1 INS
27 P2 R1 DELS
28 P2 R1 HQINS
29 P2 R1 HQDEL
30 P2 R2 AS
31 P2 R2 ED
32 P2 R2 MM
33 P2 R2 HQMM
34 P2 R2 GO
35 P2 R2 GE
36 P2 R2 INS
37 P2 R2 DELS
38 P2 R2 HQINS
39 P2 R2 HQDEL
40 Span diff
41 AS diff
42 ED diff
43 MAT diff
44 MM diff
45 HQMM diff
46 GO diff
47 GE diff
48 INS diff
49 DELS diff
50 HQINS diff
51 HQDEL diff
52 PARENT


In [6]:
class DataLoader():
    def __init__(self,filepath1,filepath2,verbose=True):
        self.files = [filepath1,filepath2]
        self.alignments=[]
        self.labels=[]
        self.verbose = verbose
        self.max_lines = None
        self.ties = 0
        self.predictions = []
        self.num_features = 1

    def set_num_features(self,count):
        self.num_features = count

    def set_max_lines(self,lines):
        '''Limit the dataset size to fit in RAM.'''
        self.max_lines = lines
        if self.verbose:
            print('Maximum lines to load per file: %d'%lines)

    def _count_lines_(self):
        '''Show number of lines per input file.'''
        count0 = 0
        with gzip.open (self.files[0],'rt') as handle0:
            for row in handle0:
                count0 += 1
        count1 = 0
        with gzip.open(self.files[1],'rt') as handle1:
            for row in handle1:
                count1 += 1
        minimum = min(count0,count1)
        if self.verbose:
            print('File0 size: %d %s'%(count0,self.files[0]))
            print('File1 size: %d %s'%(count1,self.files[1]))
        return minimum

    def _load_line_(self,row):
        '''Load data structure from one line of CSV file.'''
        line = row.strip()
        fields = line.split(',')
        ints = [0] * self.num_features
        # These fields come straight from the input file.
        # These fields are grouped by which read they describe.
        # P1 R1 = Parent 1, Read 1
        ints[0] = int(fields[0]) # P1 R1 AS
        ints[1] = int(fields[1]) # P1 R1 ED
        P1R1_MAT = int(fields[2]) # P1 R1 MAT
        ints[2] = int(fields[3]) # P1 R1 MM
        ints[3] = int(fields[4]) # P1 R1 HQMM
        ints[4] = int(fields[5]) # P1 R1 GO
        ints[5] = int(fields[6]) # P1 R1 GE
        ints[6] = int(fields[7]) # P1 R1 INS
        ints[7] = int(fields[8]) # P1 R1 DELS
        ints[8] = int(fields[9]) # P1 R1 HQINS
        ints[9] = int(fields[10]) # P1 R1 HQDEL
        #  = Parent 1, Read 2
        ints[10] = int(fields[11]) # P1 R2 AS
        ints[11] = int(fields[12]) # P1 R2 ED
        P1R2_MAT = int(fields[13]) # P1 R2 MAT
        ints[12] = int(fields[14]) # P1 R2 MM
        ints[13] = int(fields[15]) # P1 R2 HQMM
        ints[14] = int(fields[16]) # P1 R2 GO
        ints[15] = int(fields[17]) # P1 R2 GE
        ints[16] = int(fields[18]) # P1 R2 INS
        ints[17] = int(fields[19]) # P1 R2 DELS
        ints[18] = int(fields[20]) # P1 R2 HQINS
        ints[19] = int(fields[21]) # P1 R2 HQDEL
        # P2 R1 = Parent 2, Read 1
        ints[20] = int(fields[22]) # P2 R1 AS
        ints[21] = int(fields[23]) # P2 R1 ED
        P2R1_MAT = int(fields[24]) # P2 R1 MAT
        ints[22] = int(fields[25]) # P2 R1 MM
        ints[23] = int(fields[26]) # P2 R1 HQMM
        ints[24] = int(fields[27]) # P2 R1 GO
        ints[25] = int(fields[28]) # P2 R1 GE
        ints[26] = int(fields[29]) # P2 R1 INS
        ints[27] = int(fields[30]) # P2 R1 DELS
        ints[28] = int(fields[31]) # P2 R1 HQINS
        ints[29] = int(fields[32]) # P2 R1 HQDEL
        # P2 R2 = Parent 2, Read 2
        ints[30] = int(fields[33]) # P2 R2 AS
        ints[31] = int(fields[34]) # P2 R2 ED
        P2R2_MAT = int(fields[35]) # P2 R2 MAT
        ints[32] = int(fields[36]) # P2 R2 MM
        ints[33] = int(fields[37]) # P2 R2 HQMM
        ints[34] = int(fields[38]) # P2 R2 GO
        ints[35] = int(fields[39]) # P2 R2 GE
        ints[36] = int(fields[40]) # P2 R2 INS
        ints[37] = int(fields[41]) # P2 R2 DELS
        ints[38] = int(fields[42]) # P2 R2 HQINS
        ints[39] = int(fields[43]) # P2 R2 HQDEL
        # Fields that come in twos
        R1_LEN = int(fields[44]) # R1 length (of read)
        R2_LEN = int(fields[45]) # R2 length (of read)
        P1_SPAN = int(fields[46]) # P1 span (of mapped read pair)
        P2_SPAN = int(fields[47]) # P2 span (of mapped read pair)
        # Read-wise differences
        ints[40] = P2_SPAN-P1_SPAN # P2-P1 span diff
        ints[41] = (ints[33]+ints[22])-(ints[11]+ints[0]) # AS diff
        ints[42] = (ints[34]+ints[23])-(ints[12]+ints[1]) # ED diff
        ints[43] = (P2R1_MAT+P2R2_MAT)-(P1R1_MAT+P1R2_MAT) # MAT diff
        ints[44] = (ints[36]+ints[25])-(ints[14]+ints[3]) # MM diff
        ints[45] = (ints[37]+ints[26])-(ints[15]+ints[4]) # HQMM diff
        ints[46] = (ints[38]+ints[27])-(ints[16]+ints[5]) # GO diff
        ints[47] = (ints[39]+ints[28])-(ints[17]+ints[6]) # GE diff
        ints[48] = (ints[40]+ints[29])-(ints[18]+ints[7]) # INS diff
        ints[49] = (ints[41]+ints[30])-(ints[19]+ints[8]) # DELS diff
        ints[50] = (ints[42]+ints[31])-(ints[20]+ints[9]) # HQINS diff
        ints[51] = (ints[43]+ints[32])-(ints[21]+ints[10]) # HQDEL diff
        # The feature-extraction program populated a field
        # to indicate which parent had higher alignment score.
        # Values were 0=same, 1=parent1, 2=parent2.
        # We change the values to -1=parent1, 0=unknown, +1=parent2
        parent_choice = int(fields[48])
        if parent_choice == 1:
            ints[52] = -1  # not parent 2
        elif parent_choice == 2:
            ints[52] = 1  # is parent 2
        elif parent_choice == 0:
            ints[52] = 0
        else:
            raise Exception('Unrecognized parent choice:'+str(parent_choice))
        # For fair comparison, force aligner to choose.
        # We change 1 to 0, 2 to 1, and 0 to 1 or 2 randomly.
        # TO DO: faster alternative to list.append() ???
        parent_choice = int(fields[48])
        if parent_choice == 1:
            self.predictions.append(0)  # not parent 2
        elif parent_choice == 2:
            self.predictions.append(1)  # is parent 2
        else: # parent_choice == 0:
            self.ties += 1
            guess = random.randint(0,1)
            self.predictions.append(guess)
        # The transcript that this read pair aligned to.
        # This is for pipelines that only process reads that map
        # to same transcript in both parents and (filter the others).
        # Pipelines that assign reads to parent, regardless of which gene,
        # should ignore this value. (It only reflects first parent map anyway.)
        transcript_id = fields[49] # TO DO: where to put this?
        self.alignments.append(ints)

    def count_ties(self):
        return self.ties

    def load_full_train_set(self):
        '''Load full train set (to be used for train and valiation).
           Use set_max_lines() to leave some data for the test set.'''
        minimum = 0
        train_size = self.max_lines
        if self.verbose:
            print('Trying to load %d lines per file...'%train_size)
        try:
            handle0 = gzip.open(self.files[0],'rt')
            handle1 = gzip.open(self.files[1],'rt')
            # Associate label 0 with data from file 0. Same for 1.
            for i in range(train_size):
                row = next(handle0)
                self._load_line_(row)
                self.labels.append(0)
                row = next(handle1)
                self._load_line_(row)
                self.labels.append(1)
            handle0.close()
            handle1.close()
        except Exception as e:
            print(e)
            print('Most likely, one file has too few reads.')
            raise Exception('CANNOT LOAD DATA FROM FILE!')

    def show_examples(self,head=6):
        head = min(head,len(self.alignments))
        for i in range(head):
            print('From '+self.files[self.labels[i]])
            print('Score,Edit,MM,HQMM,GapOpen,GapExtend,INS,DELS,HQINS,HQDEL')
            print(self.alignments[i][0:9])
            print(self.alignments[i][10:19])
            print(self.alignments[i][20:29])
            print(self.alignments[i][30:39])
            print('Parent choice:',self.alignments[i][52])

    def get_X_y(self):
        loaded = len(self.alignments)
        divider = int(loaded - loaded * VALID_PORTION)
        X_train = np.array(self.alignments[:divider])
        y_train = np.array(self.labels[:divider])
        X_valid = np.array(self.alignments[divider:])
        y_valid = np.array(self.labels[divider:])
        if self.verbose:
            print('Full train set size = '+str(len(self.alignments)))
            print('Training/Validation partition: %d/%d'%(len(y_train),len(y_valid)))
        return X_train,y_train, X_valid,y_valid

    def get_predictions(self):
        loaded = len(self.predictions)
        divider = int(loaded - loaded * VALID_PORTION)
        y_train = self.predictions[:divider]
        y_valid = self.predictions[divider:]
        return y_train, y_valid

In [7]:
print(datetime.now())
filepath0 = DATA_DIR+DATA_FILE_0
filepath1 = DATA_DIR+DATA_FILE_1
loader=DataLoader(filepath0,filepath1)
loader.set_max_lines(MAX_LINES_TO_LOAD)
loader.set_num_features(len(feature_names))
print('LOADING')
loader.load_full_train_set()
print('Number of ties: %d' % loader.count_ties())
aligner_predictions_train, aligner_predictions_valid = loader.get_predictions()
print(datetime.now())

2023-11-16 12:57:47.660093
Maximum lines to load per file: 1000000
LOADING
Trying to load 1000000 lines per file...
Number of ties: 1672533
2023-11-16 12:58:33.114785


In [8]:
X_train,y_train, X_valid,y_valid = loader.get_X_y()
X_valid = None
y_valid = None
print('X train shape: ')
print(np.shape(X_train))
print('y train shape: ')
print(np.shape(y_train))

Full train set size = 2000000
Training/Validation partition: 1600000/400000
X train shape: 
(1600000, 53)
y train shape: 
(1600000,)


## Model

In [9]:
def build_model():
    rfc = RFC()
    return rfc

In [10]:
# docs: Note that in binary classification, recall of the positive class is also known as “sensitivity”; recall of the negative class is “specificity”.
def show_performance(y_valid, yhat_classes, yhat_pred):
    accuracy = accuracy_score(y_valid, yhat_classes)*100.
    precision = precision_score(y_valid, yhat_classes)*100.
    recall = recall_score(y_valid, yhat_classes)*100.
    sensitivity = recall_score(y_valid, yhat_classes, pos_label=1)*100.
    specificity = recall_score(y_valid, yhat_classes, pos_label=0)*100.
    f1 = f1_score(y_valid, yhat_classes)*100.
    mcc = matthews_corrcoef(y_valid, yhat_classes)
    if yhat_pred is None:
        # these stats are possible for probabilistic models only
        auprc = 0.
        auroc = 0.
    else:
        prc_Y, prc_X, prc_bins = precision_recall_curve(y_valid, yhat_pred)
        auprc = auc(prc_X,prc_Y)*100.
        auroc = roc_auc_score(y_valid, yhat_pred)*100.
    values,counts=np.unique(yhat_classes, return_counts=True)
    print('Predictions: ', dict(zip(values, counts)))
    print('Accuracy: %.2f%% F1: %.2f%% MCC: %.4f' % (accuracy,f1,mcc))
    print('Precision: %.2f%% Recall: %.2f%% AUPRC: %.2f%%' % (precision,recall,auprc))
    print('Sensitivity: %.2f%% Specificity: %.2f%% AUROC: %.2f%%' % (sensitivity,specificity,auroc))
    return accuracy,precision,recall,f1,sensitivity,specificity,mcc,auprc,auroc

## Cross Validation

In [11]:
print(datetime.now())
FOLD = 0
accuracyCV=list()
precisionCV=list()
recallCV=list()
f1CV=list()
sensitivityCV=list()
specificityCV=list()
mccCV=list()
auprcCV=list()
aurocCV=list()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X_train, y_train):
    rfc_model=build_model()
    FOLD += 1
    print('FOLD', FOLD)
    print('Training samples', len(train_index), 'Testing samples', len(test_index) )
    print('Training...')
    start_time = time.time()
    rfc_model.fit(X_train[train_index], y_train[train_index])
    end_time = time.time()
    print('Elapsed seconds:', (end_time-start_time))

    yhat_pairs=rfc_model.predict_proba(X_train[test_index])
    yhat_pred=[pair[1] for pair in yhat_pairs]
    yhat_classes=rfc_model.predict(X_train[test_index])

    print('Distrib of scores:',np.mean(yhat_pred),'mean',np.std(yhat_pred),'std')
    print('Range of scores:',np.min(yhat_pred),'to',np.max(yhat_pred))
    cm1 = confusion_matrix(y_train[test_index],yhat_classes)
    print('Confusion matrix\n',cm1)
    cm2 = confusion_matrix(y_train[test_index],yhat_classes,normalize='all')
    print('Normalized matrix\n',cm2)

    accuracy,precision,recall,f1,sensitivity,specificity,mcc,auprc,auroc=\
        show_performance(y_train[test_index], yhat_classes, yhat_pred)
    accuracyCV.append   (accuracy)
    precisionCV.append  (precision)
    recallCV.append     (recall)
    f1CV.append         (f1)
    sensitivityCV.append(sensitivity)
    specificityCV.append(specificity)
    mccCV.append        (mcc)
    auprcCV.append      (auprc)
    aurocCV.append      (auroc)
    print(datetime.now())

2023-11-16 12:58:45.419427
FOLD 1
Training samples 1280000 Testing samples 320000
Training...
Elapsed seconds: 316.3915777206421
Distrib of scores: 0.499990295943294 mean 0.17618520734944343 std
Range of scores: 0.0 to 1.0
Confusion matrix
 [[ 54821 104946]
 [ 32487 127746]]
Normalized matrix
 [[0.17131562 0.32795625]
 [0.10152187 0.39920625]]
Predictions:  {0: 87308, 1: 232692}
Accuracy: 57.05% F1: 65.02% MCC: 0.1576
Precision: 54.90% Recall: 79.73% AUPRC: 68.59%
Sensitivity: 79.73% Specificity: 34.31% AUROC: 62.41%
2023-11-16 13:04:15.977992
FOLD 2
Training samples 1280000 Testing samples 320000
Training...
Elapsed seconds: 327.5842139720917
Distrib of scores: 0.500343444243076 mean 0.17585518795184854 std
Range of scores: 0.0 to 1.0
Confusion matrix
 [[ 54781 105357]
 [ 32136 127726]]
Normalized matrix
 [[0.17119063 0.32924063]
 [0.100425   0.39914375]]
Predictions:  {0: 86917, 1: 233083}
Accuracy: 57.03% F1: 65.01% MCC: 0.1586
Precision: 54.80% Recall: 79.90% AUPRC: 66.74%
Sensitiv

In [12]:
print('accuracy   ',   np.mean(accuracyCV),    np.std(accuracyCV))
print('precision  ',  np.mean(precisionCV),   np.std(precisionCV))
print('recall     ',     np.mean(recallCV),      np.std(recallCV))
print('f1         ',         np.mean(f1CV),          np.std(f1CV))
print('sensitivity',np.mean(sensitivityCV), np.std(sensitivityCV))
print('specificity',np.mean(specificityCV), np.std(specificityCV))
print('mcc        ',        np.mean(mccCV),         np.std(mccCV))
print('auprc      ',      np.mean(auprcCV),       np.std(auprcCV))
print('auroc      ',      np.mean(aurocCV),       np.std(aurocCV))

accuracy    57.01275 0.09269919430610138
precision   54.795358493118194 0.135289176042054
recall      80.15036593043507 0.5539800953993225
f1          65.08951471763518 0.091986792321443
sensitivity 80.15036593043507 0.5539800953993225
specificity 33.87622108276844 0.6939001784430822
mcc         0.15824200404325456 0.0006932055428170806
auprc       67.78119729048203 0.860773550772637
auroc       62.35149496868295 0.0597581785460684
